# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from train import clean_data
from azureml.pipeline.core import Pipeline
import json
import pickle
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Getting available workspace
ws = Workspace.from_config()

experiment_name = 'my-automl-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
## Creating compute cluster
cluster_name = "my-automl-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes=20)
    
#check status of the current cluster
print(compute_target.get_status().serialize())


In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name

datastore= ws.get_default_datastore()

found = False
key = "Breast Cancer Dataset"
description_text = "Wisconsin Breast Cancer Dataset downloaded from Kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("found dataset")

if not found:
        # Upload csv file into default datastore
        print("uploading csv file")
        datastore.upload_files(files = ['./breast_cancer_dataset.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
        
        # Create AML Dataset and register it into Workspace
        print("creating dataset")
        ###dataset = Dataset.Tabular.from_delimited_files(datastore.path('./breast_cancer_dataset.csv'))        
        dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, './breast_cancer_dataset.csv')])
        
        #Register Dataset in Workspace
        print("registering dataset")
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
        print("created dataset")


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
## Create scoring dataset by dropping the result column
dataset_score = df.drop('diagnosis', axis=1)
dataset_score.head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
## Setting parameters for AutoMLConfig
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [ ]:
## Submitting automl run
automl_experiment = Experiment(ws,'automl_test_experiment')
automl_run = automl_experiment.submit(config=automl_config,
                                     show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
## Use RunDetails widget to show details of the experiment in progress
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == 'Completed')

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
## Retrieving best automl model
automl_best_run = automl_run.get_best_run_by_primary_metric()
automl_best_run_mertics = automl_best_run.get_metrics()
#parameter_values = automl_best_run.get_details()['runDefinition']['Arguments']
print('best run id : ', automl_best_run.id)
print('best run accuracy :', automl_best_run_metrics['Accuracy'])

'''
best_run, fitted_model = remote_run.get_output()
'''

## Saving best automl model
best_run_model_name = 'Udacity-automl-model'
best_run_model_path = 'output/model'
model = automl_best_run.register_model(model_name=best_run_model_name,
                               model_path=best_run_model_path)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
description = 'My AutoML Model'
model = automl_run.register_model(description = description)

print(automl_run.model_id)

'''    
env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)
'''

env = automl_best_run.get_environment()    

inference_config = InferenceConfig(entry_script='score.py',
                                   environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_data = json.dumps({
    'data': dataset_score[0:2].to_dict(orient= 'records')
    
})
print(input_data)

In [ ]:
output_data = webservice.run(input_data)
print(output_data)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
## Print logs
print(service.get_logs())

## Delete service
service.delete()